In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [11]:
img = cv2.imread('test2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
cv2.imshow('img', gray)
cv2.waitKey(0)

-1

In [9]:
Fy = convolve(gray, 9, 0.2)
cv2.imshow('img', Fy)
cv2.waitKey(0)

-1

In [49]:
# lines = cv2.HoughLines(Fy2, 1, np.pi/180, 624)
# lines = detectLines(Fy, minVotes = 80, rRes = 10, tRes = np.pi / 20, best = 3)
lines = detectLines(Fy, minVotes = 20, minBrightness = 10, rRes = 1, tRes = np.pi / 180, best = 1)
temp = img.copy()
print(lines)

for l in lines:
    rho, theta, votes = l
    b = np.cos(theta)
    a = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(temp,(x1,y1),(x2,y2),(0, 0, 255),2)

# cv2.imwrite('test3.jpg',Fy)
cv2.imshow('img', temp)
cv2.waitKey(0)

582


KeyboardInterrupt: 

In [52]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
Fx = convolve_x(gray, 9, 0.2)
print('hi')
lines = detectLines(Fx, minVotes = gray.shape[0] // 3, rRes = 1, tRes = np.pi / 180, best = 1)
temp = img.copy()

# if lines != None:
    
for l in lines:
    rho, theta, votes = l
    b = np.cos(theta)
    a = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(temp,(x1,y1), (x2,y2), (0,0,255), 2)

cv2.imshow('img', temp)
cv2.waitKey(0)

hi
582


-1

In [51]:
def convolve_x(img, height = 9, sigma = 1):
    Gx = np.array([1 / (sigma * np.sqrt(2 * np.pi)) * np.e**(-y**2 / (2 * sigma**2)) for y in range(-height//2+1, height//2+1)])
    Gx = np.expand_dims(Gx, axis = 0) / np.sum(Gx)
    Gx = Gx - np.ones(height) / height
    out = cv2.filter2D(img, -1, Gx)
    out[out > 30] = 255
    return out

In [27]:
def detectLines(img, minVotes = 10, rRes = 1, tRes = np.pi / 180, minBrightness = 240, best = None):
    height, width = img.shape
    ans = dict()
    print(height)
    for r in range(height):
#         if r % 100 == 0:
#             print(r)
        for c in range(width):
            if img[r,c] < minBrightness:
                continue
            for i in range(int(np.pi / tRes)):
                rho = (r * np.cos(i * tRes) + c * np.sin(i * tRes)) // rRes * rRes
                if rho in ans.keys():
                    ans[rho][i] = ans[rho].get(i, 0) + 1
                else:
                    ans[rho] = dict()
                    ans[rho][i] = 1
                    
    # create output array
    out = []
    for rho, d in ans.items():
        for theta, votes in d.items():
            if votes >= minVotes:
                out.append([rho, theta * tRes, ans[rho][theta]])
    if len(out) == 0:
        return None# detectLines(img, minVotes//3, rRes, tRes, minBrightness, best = best)
    out = np.array(out)
    
    if best == None:
        return out[(-out)[:, 2].argsort()]
    return out[(-out)[:, 2].argsort()][:best]
    
# print(detectLines(Fy, 80))